In [14]:
# Import libraries
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.heuristics import solve_tsp_local_search
import geopandas
import geoplot
import mplleaflet

In [15]:
# Load data
df = pd.read_csv("flower_input.csv", sep=";")
df

,Aanhef,Voorletters,Tussenvoegsel,Achternaam,Straatnaam,Huisnummer,Toevoeging,Postcode,Woonplaats
0,Company,NaN,NaN,Itility,Flight Forum,3360,NaN,5657 EW,Eindhoven
1,Mevr.,N.J.,NaN,Verheggen,Venuslaan,97,NaN,5632 HB,Eindhoven
2,Mevr.,M.H.C.,NaN,Verheijen,Imkerstraat,244,NaN,5623 DK,Eindhoven
3,Dhr. en mevr.,NaN,NaN,Vermeulen,Antwerpenlaan,25,NaN,5628 XC,Eindhoven
4,Mevr.,G.,NaN,Verschuren,Venuslaan,711,NaN,5632 HV,Eindhoven
...,...,...,...,...,...,...,...,...,...
105,Mevr.,NaN,NaN,Veldpaus,Luipaardstraat,82,NaN,5645 CW,Eindhoven
106,Mevr.,NaN,NaN,Veth,Roostenlaan,89,NaN,5644 GB,Eindhoven
107,Mevr.,NaN,NaN,Vloet,Permekestraat,42,NaN,5643 VX,Eindhoven
108,Mevr. + Dhr.,NaN,NaN,Wiel,Gentiaanweg,15,NaN,5643 CA,Eindhoven


In [3]:
""" Enrich the flower dataframe with correct column types and by adding columns
    - 'Name' with the full name of a person
    - 'Address' with the full address of a person
    - 'Size' with the number of flowers that need to be delivered """

# Add Name column to the flower dataset
df['Aanhef'] = df['Aanhef'].astype(str)
df['Voorletters'] = df['Voorletters'].astype(str)
df['Tussenvoegsel'] = df['Tussenvoegsel'].astype(str)
df['Achternaam'] = df['Achternaam'].astype(str)
df['Name'] = df[['Aanhef', 'Voorletters', 'Tussenvoegsel', 'Achternaam']].agg(' '.join, axis=1)
df['Name'] = df['Name'].str.replace('nan ', '')

# Add Address to the flower dataset
df['Straatnaam'] = df['Straatnaam'].astype(str)
df['Huisnummer'] = df['Huisnummer'].astype(str)
df['Postcode'] = df['Postcode'].astype(str).str.replace(' ', '')
df['Woonplaats'] = df['Woonplaats'].astype(str)
df['Address'] = df[['Straatnaam', 'Huisnummer', 'Postcode', 'Woonplaats']].agg(' '.join, axis=1)
df['Address'] = df['Address'].str.replace('nan ', '')
df

,Aanhef,Voorletters,Tussenvoegsel,Achternaam,Straatnaam,Huisnummer,Toevoeging,Postcode,Woonplaats,Name,Address
0,Company,nan,nan,Itility,Flight Forum,3360,NaN,5657EW,Eindhoven,Company Itility,Flight Forum 3360 5657EW Eindhoven
1,Mevr.,N.J.,nan,Verheggen,Venuslaan,97,NaN,5632HB,Eindhoven,Mevr. N.J. Verheggen,Venuslaan 97 5632HB Eindhoven
2,Mevr.,M.H.C.,nan,Verheijen,Imkerstraat,244,NaN,5623DK,Eindhoven,Mevr. M.H.C. Verheijen,Imkerstraat 244 5623DK Eindhoven
3,Dhr. en mevr.,nan,nan,Vermeulen,Antwerpenlaan,25,NaN,5628XC,Eindhoven,Dhr. en mevr. Vermeulen,Antwerpenlaan 25 5628XC Eindhoven
4,Mevr.,G.,nan,Verschuren,Venuslaan,711,NaN,5632HV,Eindhoven,Mevr. G. Verschuren,Venuslaan 711 5632HV Eindhoven
...,...,...,...,...,...,...,...,...,...,...,...
105,Mevr.,nan,nan,Veldpaus,Luipaardstraat,82,NaN,5645CW,Eindhoven,Mevr. Veldpaus,Luipaardstraat 82 5645CW Eindhoven
106,Mevr.,nan,nan,Veth,Roostenlaan,89,NaN,5644GB,Eindhoven,Mevr. Veth,Roostenlaan 89 5644GB Eindhoven
107,Mevr.,nan,nan,Vloet,Permekestraat,42,NaN,5643VX,Eindhoven,Mevr. Vloet,Permekestraat 42 5643VX Eindhoven
108,Mevr. + Dhr.,nan,nan,Wiel,Gentiaanweg,15,NaN,5643CA,Eindhoven,Mevr. + Dhr. Wiel,Gentiaanweg 15 5643CA Eindhoven


In [4]:
""" Transform the 'Address' column to geolocations, adding columns 'Longitude' and 'Latitude' to the dataframe """

# Initialize geolocator object
geolocator = Nominatim(user_agent="tulip_hackathon")

# Initalize a dataframe with failed addresses
fail_df = pd.DataFrame(columns=df.columns)

# Initialize longitude and latitude columns
df['Longitude'] = 0
df['Latitude'] = 0

# Loop over all addresses, and add the geolocations to the dataframe
for index, row in df.iterrows():

# Set location object
    location = geolocator.geocode(row["Address"])

# Try to add longitude and latitude columns, otherwise add entire row to fail_df
    try:
        df.loc[index, 'Longitude'] = float(location.longitude)
        df.loc[index, 'Latitude'] = float(location.latitude)
    except AttributeError:
        fail_df = fail_df.append(row, ignore_index=True)

    # Print the index to keep track of the loop
    print("Geo location conversions: " + str(index))

Geo location conversions: 0
Geo location conversions: 1
Geo location conversions: 2
Geo location conversions: 3
Geo location conversions: 4
Geo location conversions: 5
Geo location conversions: 6
Geo location conversions: 7
Geo location conversions: 8
Geo location conversions: 9
Geo location conversions: 10
Geo location conversions: 11
Geo location conversions: 12
Geo location conversions: 13
Geo location conversions: 14
Geo location conversions: 15
Geo location conversions: 16
Geo location conversions: 17
Geo location conversions: 18
Geo location conversions: 19
Geo location conversions: 20
Geo location conversions: 21
Geo location conversions: 22
Geo location conversions: 23
Geo location conversions: 24
Geo location conversions: 25
Geo location conversions: 26
Geo location conversions: 27
Geo location conversions: 28
Geo location conversions: 29
Geo location conversions: 30
Geo location conversions: 31
Geo location conversions: 32
Geo location conversions: 33
Geo location conversions

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Aanhef         110 non-null    object 
 1   Voorletters    110 non-null    object 
 2   Tussenvoegsel  110 non-null    object 
 3   Achternaam     110 non-null    object 
 4   Straatnaam     110 non-null    object 
 5   Huisnummer     110 non-null    object 
 6   Toevoeging     15 non-null     object 
 7   Postcode       110 non-null    object 
 8   Woonplaats     110 non-null    object 
 9   Name           110 non-null    object 
 10  Address        110 non-null    object 
 11  Longitude      110 non-null    float64
 12  Latitude       110 non-null    float64
dtypes: float64(2), object(11)
memory usage: 11.3+ KB


In [6]:
""" Calculate the 'as the crow flies' distance matrix for all locations in the flower dataframe """

# Initialize the distance matrix
all_locations = np.arange(0, df.shape[0])
distance_matrix = np.ones((df.shape[0], df.shape[0]))

# Fill the distance matrix with bird's distances
for i in all_locations:
    for j in all_locations:
        distance_matrix[i, j] = np.sqrt((df.loc[i, 'Longitude'] - df.loc[j, 'Longitude']) ** 2 +
                                     (df.loc[i, 'Latitude'] - df.loc[j, 'Latitude']) ** 2)

In [7]:
permutation, distance = solve_tsp_local_search(distance_matrix)

In [8]:
permutation

[0,
 59,
 80,
 83,
 84,
 90,
 86,
 87,
 89,
 91,
 85,
 92,
 88,
 82,
 71,
 74,
 66,
 72,
 48,
 65,
 57,
 43,
 56,
 52,
 55,
 53,
 58,
 49,
 44,
 46,
 47,
 45,
 54,
 50,
 100,
 103,
 99,
 106,
 108,
 104,
 93,
 105,
 97,
 101,
 102,
 94,
 107,
 96,
 95,
 98,
 109,
 76,
 78,
 77,
 75,
 9,
 16,
 4,
 11,
 5,
 6,
 8,
 33,
 3,
 15,
 28,
 18,
 19,
 31,
 30,
 23,
 36,
 24,
 32,
 17,
 34,
 26,
 35,
 20,
 21,
 10,
 14,
 1,
 13,
 12,
 7,
 41,
 2,
 38,
 39,
 40,
 37,
 67,
 25,
 29,
 27,
 51,
 63,
 64,
 42,
 69,
 73,
 61,
 68,
 60,
 70,
 62,
 81,
 79,
 22]

In [9]:
df["Stop Number"] = permutation
df_sorted = df.sort_values("Stop Number")
df_short = df_sorted[["Longitude", "Latitude", "Stop Number"]]

In [10]:
gdf = geopandas.GeoDataFrame(
    df_short, geometry=geopandas.points_from_xy(df_short.Longitude, df_short.Latitude))

In [11]:
gdf.head()

,Longitude,Latitude,Stop Number,geometry
0,5.404500,51.450187,0,POINT (5.40450 51.45019)
82,5.444673,51.438228,1,POINT (5.44467 51.43823)
87,5.439821,51.439058,2,POINT (5.43982 51.43906)
63,5.471118,51.439575,3,POINT (5.47112 51.43957)
57,5.461354,51.426620,4,POINT (5.46135 51.42662)


In [16]:
# Visualize
fig, ax = plt.subplots(figsize=(16,16))
gdf.plot(x='Longitude', y='Latitude', kind='scatter', s=40, 
                color='red', ax=ax)
fig = mplleaflet.display(fig=fig)
fig

/home/wouter/anaconda3/envs/spatial/lib/python3.7/site-packages/mplleaflet/mplexporter/exporter.py:263: MatplotlibDeprecationWarning: 
The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  offset_order = offset_dict[collection.get_offset_position()]
/home/wouter/anaconda3/envs/spatial/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
